In [2]:
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

In [3]:
zero_noise = pd.read_csv("data/zero_noise.csv")
low_noise = pd.read_csv("data/low_noise.csv")
high_noise = pd.read_csv("data/high_noise.csv")
low_noise.drop(columns=["data_type"], inplace=True)
high_noise.drop(columns=["data_type"], inplace=True)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def encode(v, class_values):
    return class_values.index(v)

def encode_target(v, class_values):
    return class_values.index(v)

class_values_zero = list(zero_noise.era.unique())
zero_noise["era_label"] = zero_noise["era"].apply(encode, args=(class_values_zero,))

class_values_low = list(low_noise.era.unique())
class_values_low_target_5 = list(low_noise.target_5_val.unique())
class_values_low_target_10 = list(low_noise.target_10_val.unique())
low_noise["era_label"] = low_noise["era"].apply(encode, args=(class_values_low,))
low_noise["target_5_val_label"] = low_noise["target_5_val"].apply(encode_target, args=(class_values_low_target_5,))
low_noise["target_10_val_label"] = low_noise["target_10_val"].apply(encode_target, args=(class_values_low_target_10,))

class_values_high = list(high_noise.era.unique())
class_values_high_target_5 = list(high_noise.target_5_val.unique())
class_values_high_target_10 = list(high_noise.target_10_val.unique())
high_noise["era_label"] = high_noise["era"].apply(encode, args=(class_values_high,))
high_noise["target_5_val_label"] = high_noise["target_5_val"].apply(encode_target, args=(class_values_high_target_5,))
high_noise["target_10_val_label"] = high_noise["target_10_val"].apply(encode_target, args=(class_values_high_target_10,))

dataset = high_noise
target_column = "target_10_val_label"
output_classes = 12 if target_column=="era_label" else 5

In [5]:
class CustomDataset(Dataset):
    def __init__(self, X, y, transform=None):
        self.X = X
        self.y = y
        self.transform = transform

    def __len__(self):
        return len(self.X)

    def __getitem__(self, index):
        return (
            torch.tensor(self.X[index], dtype=torch.float32, device=device),
            torch.tensor(self.y[index], dtype=torch.long, device=device),
        )

In [6]:
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))

dataset = dataset.sample(frac=1).reset_index(drop=True)
train_dataset = dataset.iloc[:train_size]
val_dataset = dataset.iloc[train_size:train_size+val_size]
test_dataset = dataset.iloc[train_size+val_size:]

train_X = train_dataset.iloc[:, :-8].values
train_y = train_dataset.loc[:, target_column].values

val_X = val_dataset.iloc[:, :-8].values
val_y = val_dataset.loc[:, target_column].values

test_X = test_dataset.iloc[:, :-8].values
test_y = test_dataset.loc[:, target_column].values

In [7]:
train_dataset = CustomDataset(train_X, train_y)
val_dataset = CustomDataset(val_X, val_y)
test_dataset = CustomDataset(test_X, test_y)

train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=False)
val_dataloader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [22]:
class MLP(nn.Module):
    def __init__(self, layers):
        super().__init__()
        self.layers = layers
        self.linear = nn.Linear(output_classes, 1)

    def forward(self, x):

        for layer in self.layers[:-1]:
            x = layer(x)

        logits = self.layers[-1](x).to(torch.float64).to(device)
        x1 = self.linear(x)
        tau = torch.sigmoid(x1).to(torch.float64).to(device)

        return logits, tau
    

class Autoencoder(nn.Module):
    def __init__(self, input_dim=24, encoding_dim=16):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 48),
            nn.ReLU(),
            nn.Linear(48, 32),
            nn.ReLU(),
            nn.Linear(32, encoding_dim),
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 48),
            nn.ReLU(),
            nn.Linear(48, input_dim),
        )

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

    
class NoiseAttentionLoss(nn.Module):
    def __init__(self, lambda_=1.0):
        super(NoiseAttentionLoss, self).__init__()
        self.lambda_ = lambda_

    def forward(self, logits, y, tau):
        y = F.one_hot(y, output_classes).to(torch.float64).to(device)
        perceptual = tau.squeeze(1) * (logits.t() - y.t()) + y.t()
        attention_term = torch.matmul(y, torch.log(perceptual + 1e-8))
        attention_term = attention_term.diag()
        boost_term = torch.log(tau + 1e-8) * self.lambda_

        attention_term = -torch.mean(attention_term)
        boost_term = -torch.mean(boost_term)

        return attention_term + boost_term


class SubTab(nn.Module):
    def __init__(self):
        super(SubTab, self).__init__
        

layers = nn.ModuleList(
    [
        nn.Linear(24, 64),
        nn.ReLU(),
        nn.Linear(64, 128),
        nn.ReLU(),
        nn.Linear(128, 32),
        nn.ReLU(),
        nn.Linear(32, output_classes),
        nn.Softmax(dim=1),
    ]
)

autoencoder = Autoencoder(input_dim=24, encoding_dim=16).to(device)
model = MLP(layers).to(device)

learning_rate = 0.0025
learning_rate_auto = 0.001
weight_decay = 0.0001
epochs = 50
epochs_auto = 50

# criterion = nn.CrossEntropyLoss()
criterion = NoiseAttentionLoss(lambda_=50).to(device)
criterion_auto = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer_auto = torch.optim.Adam(autoencoder.parameters(), lr=learning_rate_auto)

In [23]:
def train_autoencoder(model, dataloader, epochs, criterion, optimizer):
    model.train()
    for epoch in tqdm(range(epochs)):
        running_loss = 0.0

        for features, _ in iter(dataloader):
            input = features.to(device)
            target = features.to(device)
            optimizer.zero_grad()
            outputs = model(input)
            loss = criterion(outputs, target)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}/{epochs} - Training loss: {running_loss/len(dataloader)}")

# train_autoencoder(autoencoder, train_dataloader, epochs_auto, criterion_auto, optimizer_auto)

In [24]:
def train(model, optimizer, loss_fn, epochs, train_dataloader, val_dataloader,
          bagging_mode=False, validation_mode=True, NAL_loss=False, debug_mode=False):
    
    batch=0
    val_losses=[]
    val_accuracies=[]
    train_losses=[]
    train_accuracies=[]
    num_batches=int(len(train_dataloader)*0.9)
    for epoch in range(epochs):
        
        train_accuracy=0.0
        train_loss=0.0
        model.train()
        batch=0
        for feature ,label in iter(train_dataloader):
            optimizer.zero_grad()
            feature = feature.to(device)
            label = label.to(device)

            if NAL_loss:
                output, tau = model(feature)
                loss = loss_fn(output, label, tau)
            else:
                output, tau = model(feature)
                loss = loss_fn(output, label)

            loss.backward()
            optimizer.step()
            
            train_loss+=(loss.item())
            train_accuracy+=torch.sum(torch.argmax(output, dim=1) == label)/len(label)

            batch+=1
            if bagging_mode:
                if batch==num_batches:
                    break
            if debug_mode:
                break
            
        train_losses.append(train_loss/len(train_dataloader))
        train_accuracies.append(train_accuracy/len(train_dataloader))

        print(f"Epoch {epoch+1}/{epochs} - Training loss: {train_loss}")
        print(f"Epoch {epoch+1}/{epochs} - Training accuracy: {train_accuracy/len(train_dataloader)}")
        print("-----------------------------------------------")
        
        if validation_mode:
            val_loss, val_acc = validation(model, val_dataloader, loss_fn)
            val_losses.append(val_loss)
            val_accuracies.append(val_acc)
        batch=0

    if validation_mode:
        return train_losses, train_accuracies, val_losses, val_accuracies
    else:
        return train_losses, train_accuracies

def validation(model, val_dataloader, criterion):
    model.eval()
    val_accuracy = 0
    val_loss = 0
    with torch.no_grad():
        for feature, label in iter(val_dataloader):
            feature = feature.to(device)
            label = label.to(device)
            output = model(feature)
            val_loss += criterion(output, label).item()
            val_accuracy += torch.sum(torch.argmax(output, dim=1) == label)/len(label)
            
        print(f"Validation loss: {val_loss/len(val_dataloader)}")
        print(f"Validation accuracy: {val_accuracy/len(val_dataloader)}")

    return val_loss/len(val_dataloader), val_accuracy/len(val_dataloader)

def test(model, test_dataloader):
    model.eval()
    with torch.no_grad():
        accuracy = 0
        for feature, label in iter(test_dataloader):
            feature, label = feature.to(device), label.to(device)
            outputs, _ = model(feature)
            accuracy += torch.sum(torch.argmax(outputs, dim=1) == label)/len(label)
        print(f"Test accuracy: {accuracy/len(test_dataloader)}")

def plot(train, val, mode):
    plt.figure(figsize=(10, 5))
    plt.plot(train, label=f"Training {mode}")
    plt.plot(val, label=f"Validation {mode}")
    plt.title(f"Training and Validation {mode} Curve")
    plt.xlabel("Epochs")
    plt.ylabel(f"{mode}")
    plt.legend()
    plt.show()

In [25]:
def bagging_train(loss_fn, epochs, train_dataloader, val_dataloader):

    models=[]
    optims=[]
    schedulers=[]
    train_losses=[]
    train_accuracies=[]
    val_losses=[]
    val_accuracies=[]
    for i in range(5):
        model = MLP(layers).to(device)
        models.append(model)
        optimizer = torch.optim.Adam(model.parameters(), lr=0.003,weight_decay=0.0001)
        optims.append(optimizer)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=10, threshold=0.1, 
                                                               threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08)
        schedulers.append(scheduler)
    for epoch in tqdm(range(epochs)):
        loss=[]
        acc=[]
        for i in range (len(models)):
            model=models[i]
            optim=optims[i]
            print(f"Training model {i+1}")
            # print('***********************************************')
            train_loss, train_acc=train(model,optim,loss_fn,1,train_dataloader,
                                       val_dataloader,bagging_mode=False,validation_mode=False,
                                       NAL_loss=True,debug_mode=False)
            
            loss.append(train_loss); acc.append(train_acc)
        
        train_losses.append(sum(loss)/len(loss))
        train_accuracies.append(sum(acc)/len(acc))

        predictions=[]
        labels=[]
        losses = []
        for model in models:
            model.eval()
            accuracy = 0
            running_loss = 0
            model_predictions=[]
            model_labels=[]
            with torch.no_grad():
                for feature, label in iter(val_dataloader):
                    feature = feature.to(device)
                    label = label.to(device)
                    output, tau = model(feature)
                    prediction = torch.argmax(output, dim=1)
                    model_predictions.extend(prediction.item() for prediction in prediction)
                    model_labels.extend(label.item() for label in label)
                    loss = loss_fn(output, label, tau)
                    running_loss += loss.item()
            losses.append(running_loss)
            predictions.append(model_predictions)
            labels.append(model_labels)

        val_loss = sum(losses)/len(losses)
        final_labels=labels[0]
        final_predictions=[]
        for i in range(len(predictions[0])):
            current_predictions=[prediction[i] for prediction in predictions]
            final_predictions.append(max(set(current_predictions), key=current_predictions.count))
        accuracy = sum(1 for x,y in zip(final_predictions, final_labels) if x==y)/len(final_predictions)
        val_losses.append(val_loss)
        val_accuracies.append(accuracy)

        print("Validation")
        print(f"Epoch {epoch+1}/{epochs} - Validation loss: {val_loss}")
        print(f"Epoch {epoch+1}/{epochs} - Validation accuracy: {accuracy}")
        print("===============================================")

    return models, train_losses, train_accuracies, val_losses, val_accuracies

In [26]:
criterion = NoiseAttentionLoss(lambda_=50).to(device)
bagging_models, train_losses, train_accuracies, val_losses, val_accuracies = bagging_train(criterion, epochs, train_dataloader, val_dataloader)

  0%|          | 0/50 [00:00<?, ?it/s]

Training model 1


KeyboardInterrupt: 